In [0]:
# 01_bronze_stream_encounters_autoloader.ipynb
# SOURCE: Ingest raw encounter CSVs into the Bronze layer using Auto Loader.
# OUTPUT: `kardia_bronze.bronze_encounters` with Change Data Feed enabled.
# TRIGGER: Incremental batch; append to Delta table with fixed schema.

from pyspark.sql.types import StructType, StructField, StringType, DateType

# Table paths
BRONZE_DB               = "kardia_bronze"
BRONZE_ENCOUNTERS_TABLE = f"{BRONZE_DB}.bronze_encounters"

RAW_PATH                = "dbfs:/kardia/raw/encounters/"
BRONZE_PATH             = "dbfs:/kardia/bronze/bronze_encounters"

SCHEMA_PATH             = "dbfs:/kardia/_schemas/bronze_encounters"
CHECKPOINT_PATH         = "dbfs:/kardia/_checkpoints/bronze_encounters"
BAD_PATH                = "dbfs:/kardia/_quarantine/raw/bad_encounters"

# (DEMO ONLY) Minimize shuffle overhead for small test datasets
spark.conf.set("spark.sql.shuffle.partitions", "1")

In [0]:
# Define explicit schema to enforce structure and improve Auto-loader performance.
encounters_schema = StructType(
    [
        StructField("ID",                  StringType(),   False),
        StructField("DATE",                DateType(),     True),
        StructField("PATIENT",             StringType(),   False),
        StructField("CODE",                StringType(),   True),
        StructField("DESCRIPTION",         StringType(),   True),
        StructField("REASONCODE",          StringType(),   True),
        StructField("REASONDESCRIPTION",   StringType(),   True)
    ]
)

In [0]:
# 1. Ensure the Bronze DB and Bronze Encounters table exist.
spark.sql(f"CREATE DATABASE IF NOT EXISTS {BRONZE_DB}")

spark.sql(
    f"""
    CREATE TABLE IF NOT EXISTS {BRONZE_ENCOUNTERS_TABLE}
    USING DELTA
    COMMENT 'Bronze table for batch Auto Loader ingest of encounter records.'
    LOCATION '{BRONZE_PATH}'
    TBLPROPERTIES ('delta.enableChangeDataFeed' = 'true')
    """
)

In [0]:
# 2. Define an incremental batch pipeline using Auto-loader.
#    Auto-loader discovers new files in cloud storage and writes to a Delta table.
stream = (
    spark.readStream
         .format("cloudFiles")
         .option("cloudFiles.format", "csv")
         .option("cloudFiles.schemaLocation", SCHEMA_PATH)
         .option("cloudFiles.includeExistingFiles", "true")
         .option("header", "true")
         .option("badRecordsPath", BAD_PATH)
         .option("rescuedDataColumn", "_rest")
         .schema(encounters_schema)
         .load(RAW_PATH)

        .writeStream
        .format("delta")
        .option("mergeSchema", "true")
        .option("checkpointLocation", CHECKPOINT_PATH)
        .outputMode("append")
        .trigger(availableNow=True)
        .start(BRONZE_PATH)
)

stream.awaitTermination()
print(f"Bronze ingest complete: from {RAW_PATH} to {BRONZE_PATH}")

# NOTE:
# Auto Loader handles the read-side logic, maintaining a file-discovery ledger.
# Structured Streaming handles the write-side logic, using a WAL and offset tracking.
# The ledger, WAL, offsets, and schema evolution log are all stored in the checkpoint directory.
# `cloudFiles.schemaLocation` tells Auto Loader where to persist the evolving schema history.

In [0]:
# 3. Stream finished - Verify Bronze table and ingestion history.
print(f"Loaded data from {RAW_PATH} to {BRONZE_PATH}")

# Read the Bronze Encounters table into a DataFrame.
df = spark.read.format("delta").load(BRONZE_PATH)
print(f"Row count: {df.count()}")
display(df.limit(5))

# Display Delta Lake history to verify CDF and ingest details.
history_df = spark.sql(
    f"""
    DESCRIBE HISTORY delta.`{BRONZE_PATH}`
    """
).select("version", "timestamp", "operation", "operationParameters")

display(history_df.limit(5))